# EOF Analysis of AR days

* Multivariate EOF analysis - testing different configurations to see which cEOF captures the most varaince of precipitation related to AR events

  1. cEOF 250 hPa H, U, V in extratropics, 850 hPa Q, U, V in tropics
  2. cEOF 500 hPa H, U, V in extratropics, 500 hPa Q, U, V in tropics
  3. cEOF 500 hPa H, U, V in extratropics, 500 hPa H, U, V in tropics
  4. cEOF 500 hPa H, U, V in full domain [0-120 E, 0-50N]
  5. cEOF IVTu and IVTv in full domain
  6. cEOF 500 hPa H, U, V and 850 hPa Q in full domain?


In [ ]:
# Import Python modules
import os, sys
import yaml
from pathlib import Path
import numpy as np
import numpy.ma as ma
import pandas as  pd
import xarray as xr
from sklearn.cluster import KMeans
from itertools import cycle
# matplotlib
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from mpl_toolkits.axes_grid1 import AxesGrid
from matplotlib.colors import ListedColormap
from matplotlib import rcParams
import matplotlib.ticker as mticker
# cartopy
import cartopy.crs as ccrs
from cartopy.mpl.geoaxes import GeoAxes
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.feature as cfeature
# plot styles/formatting
import seaborn as sns
import cmocean.cm as cmo
import cmocean

from IPython.display import Image, display
from scipy import stats

# Path to modules
sys.path.append('../modules')

# Import my modules
from plotter import draw_basemap
from timeseries import persistence, select_months
from eofs import *
from ar_funcs import preprocess_ar_area_subregions
from kmeans import *
import nclcmaps as ncl

In [ ]:
# Set up paths

path_to_data = '/home/nash/DATA/data/'                            # project data -- read only
path_to_out  = '/home/nash/DATA/repositories/AR_types/out/'       # output files (numerical results, intermediate datafiles) -- read & write
path_to_figs = '/home/nash/DATA/repositories/AR_types/figs/'      # figures


In [ ]:
# Set a default font for all matplotlib text (can only set this ONCE; must restart kernel to change it)

rcParams['font.family'] = 'sans-serif'   # set the default font family to 'sans-serif'
rcParams['font.sans-serif'] = 'Arial'    # set the default sans-serif font to 'Arial'

## Data

### AR time series

In [ ]:
## Select MERRA2 or ERA5
reanalysis = 'era5'

if reanalysis == 'era5':
    start_date = '1979-01-01'
    end_date = '2018-12-31'
    filename = 'ar_catalog_v3_ERAI_fraction_HASIAsubregions.nc'
## if MERRA2
else:
    start_date = '1980-01-01'
    end_date = '2017-12-31'
    filename = 'ar_catalog_fraction_HASIAsubregions.nc'
    
f1 = path_to_data + 'CH1_generated_data/' + filename
ds = xr.open_dataset(f1)
# Set dates
ds = ds.sel(time=slice(start_date, end_date))
## Preprocess AR subregions - get dataframe of AR days based on area threshold
df = preprocess_ar_area_subregions(df=ds.to_dataframe(), thres=0.3)
# Show table
df.head()

### Configuration Dictionary

In [ ]:
# import configuration file for season dictionary choice
yaml_doc = '../data/config.yml'
config1 = yaml.load(open(yaml_doc), Loader=yaml.SafeLoader)
ssn = 'djfmam'
ar_dict = config1[ssn]

# import configuration file for ceof dictionary choice
## test number - NOTE as of 7/1/2020 sticking with test 4
yaml_doc = '../data/ceof_config.yml'
config2 = yaml.load(open(yaml_doc), Loader=yaml.SafeLoader)
config_dict = config2['test4']


### Import reanalysis

#### ERA5

In [ ]:
## Set variable names (for saving data/figs)
testname = config_dict['name']
eofmode = 't' # s or t
dispmat = 'cor' # dispersion matrix type correlation/covariance
pathvar = config_dict['pathvar']
ceofid = config_dict['fnameID']

# Select lat/lon grid 
# Tropics/Extratropics Domain
lonmin = config_dict['latlon'][0]
lonmax = config_dict['latlon'][1]
latmin = config_dict['latlon'][2]
latmax = config_dict['latlon'][3]

lev = config_dict['levs']

out_path = path_to_out + ceofid + '/' + ssn + '/'
fig_path = path_to_figs + ceofid + '/' + ssn + '/'

if pathvar == 'huvq':
    def preprocess(ds):
        '''keep only selected lats and lons'''
        return ds.sel(latitude=slice(latmax, latmin), longitude=slice(lonmin, lonmax), level=lev)
if pathvar == 'ivt':
        def preprocess(ds):
            '''keep only selected lats and lons'''
            return ds.sel(latitude=slice(latmax, latmin), longitude=slice(lonmin, lonmax))

# open anomaly data
filepath_pattern = path_to_data + 'ERA5/{0}/anomalies/daily_filtered_anomalies_{0}_*.nc'.format(pathvar)
    
    
f2 = xr.open_mfdataset(filepath_pattern, preprocess=preprocess, combine='by_coords')

f2

In [ ]:
if pathvar == 'huvq':

    # Create new dataset to rename lat lon
    ds = xr.Dataset({'H': (['time', 'level', 'lat', 'lon'], f2['z'].values),
                     'U': (['time', 'level',  'lat', 'lon'], f2['u'].values),
                     'V': (['time', 'level',  'lat', 'lon'], f2['v'].values),
                     'QV': (['time', 'level',  'lat', 'lon'], f2['q'].values)},
                          coords={'time': (['time'], f2.time.values),
                                  'level': (['level'], f2.level.values),
                                  'lat': (['lat'], f2.latitude.values),
                                  'lon': (['lon'], f2.longitude.values)})

if pathvar == 'ivt':
    ds = xr.Dataset({'ivte': (['time', 'lat', 'lon'], f2['p71.162'].values),
                     'ivtn': (['time', 'lat', 'lon'], f2['p72.162'].values)},
                          coords={'time': (['time'], f2.time.values),
                                  'lat': (['lat'], f2.latitude.values),
                                  'lon': (['lon'], f2.longitude.values)})


ds
print('ds size in GB {:0.2f}\n'.format(ds.nbytes / 1e9))

In [ ]:
# Add AR time series to merra; set as coordinate variables
ds['ar'] = ('time', df.ar)
ds = ds.set_coords('ar')

ds['location'] = ('time', df.location)
ds = ds.set_coords('location')

# print dataset
print(ds)

### Data Subset Selection

In [ ]:
idx = slice(ar_dict['start_date'], ar_dict['end_date'])
ds = ds.sel(time=idx)

# Select months
if ar_dict['mon_s'] > ar_dict['mon_e']:
    idx = (ds.time.dt.month >= ar_dict['mon_s']) | (ds.time.dt.month <= ar_dict['mon_e'])
else:
    idx = (ds.time.dt.month >= ar_dict['mon_s']) & (ds.time.dt.month <= ar_dict['mon_e'])

ds = ds.sel(time=idx)

# # Select AR days JUST IN R01
# idx = (ds.ar >= 1) & (ds.location == 'R01')
# Select AR days in all subregions
idx = (ds.ar >= 1)
ds_ar = ds.sel(time=idx)
# ds_ar = ds

# print results
print(ds_ar)

In [ ]:
# Count number of independent AR events and their duration in days
x = ds.ar
event_id, total_events, duration = persistence(x)
print('Total number of AR days in season: ', x.sum())
print('Total number of independent AR events: ', total_events)

## Preprocessing

### Reshape, center, and standardize data matrix

In [ ]:
%%time
# Load dataset into memory
ds_ar = ds_ar.load()


In [ ]:
## Create list of variable arrays
    
var_list = []
for i in range(len(config_dict['varlist'])):
    var1 = config_dict['varlist'][i]
    domain = config_dict['domain'][i]
    domain_bnds = config2['eof_domain'][domain]
    lonslice = slice(domain_bnds[0], domain_bnds[1])
    latslice = slice(domain_bnds[3], domain_bnds[2])
    if pathvar == 'huvq':
        lev = config_dict['varlev'][i]
        var_list.append(ds_ar[var1].sel(lon=lonslice, lat=latslice, level=lev))
    if pathvar == 'ivt':
        var_list.append(ds_ar[var1].sel(lon=lonslice, lat=latslice))

# Check that sizes of arrays match
for i, in_array in enumerate(var_list):
    # Extract variable as numpy array
    var1 = in_array.values
    print(var1.shape)


In [ ]:
%%time
# Weight the data by the square root of the cosine of the lat
var_list = spatial_weights(var_list)

## Flatten data to [time x space]
var_list = flatten_array(var_list)

## Center data
var_list = center_data(var_list)

## Standardize Arrays by removing the mean and dividing by the standard deviation of the columns
## For multivariate, place into single flattened array
Xs = standardize_arrays(var_list, mode=eofmode, dispersion_matrix=dispmat)


## EOF Analysis

In [ ]:
%%time

# Compute eigenvalues & eigenvectors
R, evals, evecs = calc_eigs(z=Xs, mode=eofmode)

print('Eigenvalues: ', evals.shape)
print(evals, '\n')

print('Eigenvectors: ', evecs.shape)
print(np.round(evecs, 3), '\n')

# save eigenvalues and eigenvectors
outfile = out_path + 'eigenvalues_' + eofmode + dispmat + '.txt'
np.savetxt(outfile, evals, fmt='%.5f')

outfile = out_path + 'eigenvectors_' + eofmode + dispmat + '.txt'
np.savetxt(outfile, evecs, fmt='%.5f', delimiter=',')

### Explained Variance

In [ ]:
# Calculate the percent explained var by each eigenvector
pctvar = pct_variance(evals)

# Number of EOFs that explain more than 1% of the total variance
idx = pctvar[pctvar >= 1.0]
neofs = len(idx)

# print exp var >= 1.0
cumvar = np.sum(pctvar[0:neofs-1])
print(f'Cumulative variance explained by the first {neofs} EOFs:')
print(f'{cumvar:.2f}% \n')

# print exp var: neofs = 4
cumvar = np.sum(pctvar[0:3])
print(f'Cumulative variance explained by the first 4 EOFs:')
print(f'{cumvar:.2f}% \n')

# print exp var for 4 eofs
for k in range(4):
    print(f'{k+1} \t {pctvar[k]:.2f}%')

### North Test

In [ ]:
err = north_test(evals, total_events)
upper = pctvar + err
lower = pctvar - err

print(np.round(upper[0:6],3))
print(np.round(pctvar[0:6],3))
print(np.round(lower[0:6],3))

In [ ]:
# get the number of EOFs that are separated based on the North Test
n = []
i = 0
while lower[i] > upper[i+1]:
    n.append(i+1)
    i += 1
print('Number of separated EOFs: ', max(n))

### EOFs and PCs

In [ ]:
# choose neofs and npcs based on North Test - where separated
# to save, plot, etc.
neofs = max(n)
npcs = neofs

# Calculate EOFS (spatial modes)
eofs = calc_eofs(Xs, evecs, evals, neofs, mode=eofmode)

# Split eofs into separate arrays for each variable
ntim, nlat, nlon = var1.shape
npts = nlat*nlon
nvar = len(var_list)
# Reshape spatial dim back to 2D map
eofmodes = var_list
for i in np.arange(len(var_list)):
    tmp = eofs[:,i*npts:(i+1)*npts]
    eofmodes[i] = np.reshape(tmp, (neofs,nlat,nlon))
    
# Calculate PCs (time coefficients)
pcs = calc_pcs(Xs, evecs, evals, npcs, mode=eofmode)
# results in [ntim, npcs] to plot in PC plot

## loadings*
## in the case of t-mode these are our "pcs" or time-coefficients
## in the case of s-mode, these are our "eofs" or spatial loadings
loads = pcs



### Spatial Modes

In [ ]:
# Data for plotting 
lons = ds_ar.lon.data
lats = ds_ar.lat.data
udat = eofmodes[1][0:neofs,:,:]
vdat = eofmodes[2][0:neofs,:,:]
data = eofmodes[0][0:neofs,:,:]

print(np.nanmin(data), np.nanmax(data))

# Set up projection
mapcrs = ccrs.PlateCarree()
datacrs = ccrs.PlateCarree()

# Set tick/grid locations
dx = np.arange(lonmin,lonmax+20,20)
dy = np.arange(latmin,latmax+20,20)

# subtitles
eof_label = [ ]
pc_label = [ ]
var_label = [ ]
for k in range(neofs):
    eof_label.append("EOF{:1d}".format(k+1,))
    pc_label.append("PC{:1d}".format(k+1,))
    var_label.append("{:.2f}%".format(pctvar[k]))

In [ ]:
# make a dict of the first n eofs with column labels for df
values = []
for i in range(neofs):
    values.append(loads[:,i])
    
keys = eof_label
dicts = dict(zip(keys, values))
# print(dicts)

# Create new dataframe
dates_arDays = ds_ar.time.values
df_out = pd.DataFrame(dicts, index=dates_arDays)

# Save loadings
outfile = out_path + 'loadings_' + eofmode + dispmat + '.csv'
# np.savetxt(outfile, loads[:,0:neofs], fmt='%.4f', delimiter=',')
df_out.to_csv(outfile)

In [ ]:
# Create figure
fig = plt.figure(figsize=(10,11))
filepath = fig_path + 'spatial_' + eofmode + dispmat + '.png'
nrows = neofs
ncols = 1

# Set up Axes Grid
axes_class = (GeoAxes,dict(map_projection=mapcrs))
axgr = AxesGrid(fig, 
                111, 
                axes_class=axes_class,
                nrows_ncols=(nrows, ncols), 
                axes_pad = 0.55,
                cbar_location='bottom', 
                cbar_mode='single',
                cbar_pad=0.0, 
                cbar_size='5%',
                label_mode='')

# Loop for drawing each plot
for k, ax in enumerate(axgr):
    ax = draw_basemap(ax, extent=[lonmin,lonmax,latmin,latmax], xticks=dx, yticks=dy)
    
    # Contour Filled
    clevs = np.arange(-30,31,5)
    cf = ax.contourf(lons, lats, data[k,:,:], transform=datacrs,
                     levels=clevs,
                     cmap="bwr", extend='both')
    # Wind barbs / vectors
    ax.quiver(lons, lats, udat[k,:,:], vdat[k,:,:], transform=datacrs,
              color='black', pivot='middle', regrid_shape=20) 
    
    # subtitles
    ax.set_title(eof_label[k], loc='left', fontsize=12)
    ax.set_title(var_label[k], loc='right', fontsize=12)
    
# single colorbar
cb = fig.colorbar(cf, axgr.cbar_axes[0], orientation='horizontal', drawedges=True)
cb.set_label('m', fontsize=11)
cb.ax.tick_params(labelsize=10)
    
# Display figure
plt.savefig(filepath, dpi=200, bbox_inches='tight')
plt.show()

### K-means clustering

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, silhouette_samples

# FUNCTIONS

def plot_optimal_k(data, neofs, kmax, create_plot=False, filename=None):
    """ Elbow plot to determine optimal number of clusters (k) 
    
    Parameters
    ----------
    data : array_like
        data to perform clustering on where rows= obs and cols = variables
    kmax : scalar, int
        maximum number of clusters; iteration over the interval k=[1,kmax]
    filename : string (optional)
        filename or path to save figure. Default is None (fig not saved).
    """
    
    # Cohesion = sum of sq dist of samples to their cluster center
    kclusters = np.arange(1, kmax, dtype=int) + 1
    cohesion = np.empty((neofs-1, kmax-1))
    silhouette = np.empty((neofs-1, kmax-1))
    upper = np.empty((neofs-1, kmax-1))
    lower = np.empty((neofs-1, kmax-1))
    
    # iterate over EOFs
    for i in range(neofs-1):
        X = data[:,0:i+2]
        
        # iterate over kclusters
        for j in range(len(kclusters)):
            ki = kclusters[j]
            km = KMeans(n_clusters=kclusters[j])
            cluster_labels = km.fit_predict(X)
            km = km.fit(X)
            cohesion[i, j] = (km.inertia_)

            # The silhouette_score gives the average value for all the samples.
            # This gives a perspective into the density and separation of the formed
            # clusters
            silhouette[i, j] = silhouette_score(X, cluster_labels)
            # Compute the silhouette scores for each sample
            sample_silhouette_values = silhouette_samples(X, cluster_labels)
            
            spread = []
            for m in range(ki):
                # Aggregate the silhouette scores for samples belonging to
                # cluster i, and sort them
                ith_cluster_silhouette_values = \
                    sample_silhouette_values[cluster_labels == m]

                ith_cluster_silhouette_values.sort()
                spread.append(ith_cluster_silhouette_values.max())
            
            lower[i, j] = min(spread)
            upper[i, j] = max(spread)

            
    if create_plot == True:
        fig = plt.figure()
        fig.set_size_inches((6.0,10.0))
        fig.dpi = 300
        sns.set_style("whitegrid")
        colors = ['tab:blue', 'tab:red', 'tab:green', 'k']
        
        # Elbow plot
        ax = plt.subplot(2, 1, 1)
        for k in range(neofs-1):
            coh = cohesion[k, :]
            ax.plot(kclusters, coh, c=colors[k], marker='o', linewidth=2.0, markersize=7.0)
#             ax.set_title('Elbow Plot for Optimal K')
            ax.set_ylabel('Sum of Sq Dist (cohesion)')
            ax.set_xlabel('k (# of clusters)')
            ax.set_xlim(1., kmax+1)
            ax.set_ylim(0, 210)
            ax.set_xticks(kclusters)
        
        # Silhouette plot
        ax2 = plt.subplot(2, 1, 2)
        for k in range(neofs-1):
            sil = silhouette[k, :]
            ax2.plot(kclusters, sil, c=colors[k], marker='o', linewidth=2.0, markersize=7.0)
            ax2.fill_between(kclusters, lower[k, :], upper[k, :], fc=colors[k], ec=None, alpha=0.2)
            ax2.set_ylabel('Mean Silhouette')
            ax2.set_xlabel('k (# of clusters)')
            ax2.set_xlim(1., kmax+1)
            ax2.set_ylim(0, 1.)
            ax2.set_xticks(kclusters)
        
        if filename:
            # Save the figure
            fig.savefig(filename, bbox_inches='tight', dpi=fig.dpi)
            fig.clf()
    else:
        return kclusters, cohesion


In [ ]:
# Determine optimal K
# maximum number of clusters (number of iterations)
kmax = 15

# Elbow plot
outfile = fig_path + 'elbow_' + eofmode + dispmat + '.png'
plot_optimal_k(loads, neofs, kmax, create_plot=True, filename=outfile)

print(outfile)
display(Image(outfile))

In [ ]:
# determine optimal k by examining the kde of the eofs
g = sns.PairGrid(df_out, diag_sharey=True, corner=True)
# g.map_upper(sns.kdeplot)
g.map_lower(sns.kdeplot)
g.map_diag(sns.kdeplot, lw = 3)
g = g.add_legend(fontsize=14)

outfile = fig_path + 'hist_kde_' + eofmode + dispmat + '.png'
g.savefig(outfile)